# Lesson 2: Pydantic Basics

In this lesson, you'll learn the fundamentals of Pydantic models for data validation using a customer support system as your example application. You'll see how to define data models, validate user input, and handle validation errors gracefully.

By the end of this lesson, you'll be able to:
- Create Pydantic models to validate user input data
- Handle validation errors with proper error handling
- Use optional fields and field constraints in your models
- Work with JSON data validation methods

---

In [1]:
# Import libraries needed for the lesson
from pydantic import BaseModel, ValidationError, EmailStr
import json

### Define a UserInput Pydantic model and populate it with data

In [2]:
# Create a Pydantic model for validating user input
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str

In [3]:
# Create a model instance
user_input = UserInput(
    name="Joe User", 
    email="joe.user@example.com", 
    query="I forgot my password."
)
print(user_input)

name='Joe User' email='joe.user@example.com' query='I forgot my password.'


### Note: the following cell will produce a validation error. You can correct the error by following along with the video, or just proceed with the rest of the notebook as cells below do not depend on this cell. 

In [4]:
# Attempt to create another model instance with an invalid email
user_input = UserInput(
    name="Joe User", 
    email="not-an-email", 
    query="I forgot my password."
)
print(user_input)

ValidationError: 1 validation error for UserInput
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not-an-email', input_type=str]

### Define a function for error handling and try different inputs

In [5]:
# Define a function to handle user input validation safely
def validate_user_input(input_data):
    try:
        # Attempt to create a UserInput model instance from user input data
        user_input = UserInput(**input_data)
        print(f"✅ Valid user input created:")
        print(f"{user_input.model_dump_json(indent=2)}")
        return user_input
    except ValidationError as e:
        # Capture and display validation errors in a readable format
        print(f"❌ Validation error occurred:")
        for error in e.errors():
            print(f"  - {error['loc'][0]}: {error['msg']}")
        return None

In [6]:
# Create an instance of UserInput using validate_user_input() function
input_data = {
    "name": "Joe User", 
    "email": "joe.user@example.com",
    "query": "I forgot my password."
}

user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I forgot my password."
}


In [7]:
# Attempt to create an instance of UserInput with missing query field
input_data = {
    "name": "Joe User", 
    "email": "joe.user@example.com"
}

user_input = validate_user_input(input_data)

❌ Validation error occurred:
  - query: Field required


### Update your UserInput data model with additional fields and experiment with different input data

In [8]:
# Import additional libraries for enhanced validation
from pydantic import Field
from typing import Optional
from datetime import date

# Define a new UserInput model with optional fields
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        None,
        description="5-digit order number (cannot start with 0)",
        ge=10000,
        le=99999
    )
    purchase_date: Optional[date] = None

In [9]:
# Define a dictionary with required fields only
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I forgot my password."
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I forgot my password.",
  "order_id": null,
  "purchase_date": null
}


In [10]:
print(user_input)

name='Joe User' email='joe.user@example.com' query='I forgot my password.' order_id=None purchase_date=None


In [11]:
# Define a dictionary with all fields including optional ones
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": date(2025, 12, 31)
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [12]:
# Define a dictionary with all fields and including additional ones
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": date(2025, 12, 31),
    "system_message": "logging status regarding order processing...",
    "iteration": 1 
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [13]:
print(user_input)

name='Joe User' email='joe.user@example.com' query='I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.' order_id=12345 purchase_date=datetime.date(2025, 12, 31)


In [14]:
# Create an instance of UserInput with valid data
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": "2025-12-31"
}

user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [15]:
# Define order_id as a string
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": "12345",
    "purchase_date": "2025-12-31"
}

# Validate the user input data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a laptop carrying case and it turned out to be \n             the wrong size. I need to return it.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [16]:
# Define name field as an integer
input_data = {
    "name": 99999,
    "email": "joe.user@example.com",
    "query": f"""I bought a laptop carrying case and it turned out to be 
             the wrong size. I need to return it.""",
    "order_id": 12345,
    "purchase_date": "2025-12-31"
}

# Validate the user input data
user_input = validate_user_input(input_data)

❌ Validation error occurred:
  - name: Input should be a valid string


### Try starting with JSON data as input

In [17]:
# Define user input as JSON data
json_data = '''
{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I bought a keyboard and mouse and was overcharged.",
    "order_id": 12345,
    "purchase_date": "2025-12-31"
}
'''

# Parse the JSON string into a Python dictionary
input_data = json.loads(json_data)
print("Parsed JSON:", input_data)

Parsed JSON: {'name': 'Joe User', 'email': 'joe.user@example.com', 'query': 'I bought a keyboard and mouse and was overcharged.', 'order_id': 12345, 'purchase_date': '2025-12-31'}


In [18]:
# Validate the user iput data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a keyboard and mouse and was overcharged.",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [19]:
# Try different JSON input
json_data = '''
{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "My account has been locked for some reason.",
    "order_id": "01234",
    "purchase_date": "2025-12-31"
}
'''

# Parse the JSON into a Python dictionary
input_data = json.loads(json_data)
print("Parsed JSON:", input_data)

Parsed JSON: {'name': 'Joe User', 'email': 'joe.user@example.com', 'query': 'My account has been locked for some reason.', 'order_id': '01234', 'purchase_date': '2025-12-31'}


In [20]:
# Validate the customer support data from JSON with non-standard formats
user_input = validate_user_input(input_data)

❌ Validation error occurred:
  - order_id: Input should be greater than or equal to 10000


### Try the `model_validate_json` method

### Note: the following cell will produce a validation error. You can correct the error by following along with the video. 

In [21]:
# Parse JSON and validate user input data in one step using model_validate_json method
user_input = UserInput.model_validate_json(json_data)
print(user_input.model_dump_json(indent=2))

ValidationError: 1 validation error for UserInput
order_id
  Input should be greater than or equal to 10000 [type=greater_than_equal, input_value='01234', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal

---

## Conclusion

In this lesson, you learned how to use Pydantic models to validate user input for a customer support scenario. By defining clear data models and handling validation errors, you can ensure your code only works with well-formed data. This approach helps you build more robust and reliable applications, and sets the stage for more advanced validation and structured output in future lessons.